In [52]:
import argparse
import mysql.connector
from astropy.table import Table
from astropy.time import Time
import os

In [67]:
def kill_sleeping_processes():
    '''WARNING
    This is dangerous
    Magic number 1800
    Magic user `nana`
    '''
    
    conn = get_db_connection()
    cursor = conn.cursor()
    query1 = 'SELECT Id FROM INFORMATION_SCHEMA.PROCESSLIST WHERE Command = "Sleep" AND User = "nana" AND Time > 1800;'
    cursor.execute(query1)

    result = cursor.fetchall()
    sleeping_ids = [row[0] for row in result]
    for sid in sleeping_ids:
        query2 = f"KILL {sid};"
        cursor.execute(query2)
        print(f"Killed sleeping process with ID {sid}")
    
    conn.commit()
    cursor.close()
    conn.close()

In [66]:
kill_sleeping_processes()

Killed sleeping process with ID 209654


In [3]:
load_dataset_table()

In [4]:
load_star_table()

In [5]:
load_task_table()

In [57]:
def start_one_task():
    #run two queries
    #ask for unstarted task, mark the task as started
   
    conn = get_db_connection()
    cursor = conn.cursor()
    query1 = "SELECT star_id, dataset_id FROM task WHERE started IS NULL LIMIT 1;"
    cursor.execute(query1)
    star_id, dataset_id = cursor.fetchall()[0]
       
    query2 = f"""UPDATE task SET 
    started = "{Time(Time.now(), format = "isot")}", 
    process_id = {os.getpid()} WHERE star_id = "{star_id}" AND dataset_id = "{dataset_id}";"""
    cursor.execute(query2)
    conn.commit()
    cursor.close()
    conn.close()
    return star_id, dataset_id

def end_one_task(star_id, dataset_id):
    query = f"""UPDATE task SET 
    finished = "{Time(Time.now(), format = "isot")}" 
    WHERE star_id = "{star_id}" AND dataset_id = "{dataset_id}";"""
    execute_query_and_close(query)

In [56]:
foo = start_one_process()
print(foo)

('KIC000757099', 'Kepler_long')


In [58]:
end_one_task(*foo)

In [6]:
assert False

AssertionError: 

In [ ]:
def kill_ghost_process():
    query = "SHOW PROCESSLIST"
    conn = get_db_connection()
    cursor = conn.cursor()
    cursur.execute(query)

    proc



In [ ]:
'''database update commands'''

def track_status(star_id, dataset_id, status, process_id=None):
    conn = mysql.connector.connect(
        host='localhost',
        user='nana',
        database='stars_db'
    )
    cursor = conn.cursor()
    if status == 'started':
        cursor.execute(
            """
            INSERT INTO task (star_id, dataset_id, started)
            VALUES (%s, %s, NOW())
            ON DUPLICATE KEY UPDATE started=NOW()
            """,
            (star_id, dataset_id)
        )
        print(f"Started task for star_id={star_id}, dataset_id={dataset_id}")

    elif status == 'finished':
        cursor.execute(
            "UPDATE task SET finished=NOW() WHERE star_id=%s AND dataset_id=%s",
            (star_id, dataset_id)
        )
        print(f"Finished task for star_id={star_id}, dataset_id={dataset_id}")
    conn.commit()
    cursor.close()
    conn.close()

def record_star(star_id, dataset_id):
    conn = mysql.connector.connect(
        host='localhost',
        user='nana',
        database='stars_db'
    )
    cursor = conn.cursor()
    cursor.execute(
        "INSERT INTO star (star_id, dataset_id) VALUES (%s, %s) "
        "ON DUPLICATE KEY UPDATE dataset_id=%s",
        (star_id, dataset_id, dataset_id)
    )
    conn.commit()
    cursor.close()
    conn.close()

 
# def record_mode(star_id, dataset_id, frequency):

def main():
    parser = argparse.ArgumentParser(description="Process a star.")
    parser.add_argument("star_id", type=str)
    parser.add_argument("--dataset_id", type=str, default="Kepler_LC")
    args = parser.parse_args()

    star_id = args.star_id
    dataset_id = args.dataset_id

    track_status(star_id, dataset_id, 'started')

    success = find_modes_in_star(star_id, plots=False, save=False, max_peaks=100)

    if success:  
        record_star(star_id, dataset_id)
        #record_mode(star_id, dataset_id)

    track_status(star_id, dataset_id, 'finished')


if __name__ == "__main__":
    main()
